In [1]:
import sys
sys.path.append("../../POMPSBO/")
from pomps.fcm import FunctionalCausalModel
from pomis.scm import Domain, RealDomain
import pyro
import torch
import pyro.distributions as dist
from pomps.fcm import FunctionalCausalModel, Functor
import typing as tp
import pandas as pd

In [2]:
def latents():
    u1 = pyro.sample("error5", dist.Uniform(-1, 1))
    u2 = pyro.sample("error2", dist.Uniform(-1, 1))
    u3 = pyro.sample("app_views_count", dist.Poisson(100))
    return {"error5": u1, 'error2': u2, 'app_views_count': u3}#, "U2": u2}

In [4]:
std = 0.42

def country_to_con_t_logits(country):
    return -0.9362607 + 0.4905761*country

def country_to_day_p_logits(country):
    return 0.1089 + 0.9141*country

def to_device(country, connection_type):
    return -0.5022 -1.5922*country -0.1550*connection_type

def to_exchange(device, day_part, country):
    return -34045.091*device -2973.584*day_part + 54712.746*country -34045.091*(device**2) + 365.688*device*day_part -53906.752*device*country + 570.279*(day_part**2) + 1667.335*day_part*country + 54712.746*(country**2)

def to_budget(country, exchange):
     return 397.04*country + 0.00375*exchange + 397.042*(country**2) + 0.00043*country*exchange -5.9e-09*(exchange**2) 
    
def to_ad_type(category, connection_type, exchange):
    return 0.8619 + 2.2977*exchange -0.3002*connection_type + 2.8386*category

def to_creative(category, device, ad_type, error5):
    return -0.00196*category -8980.50169*device + 53.20869*ad_type -0.0*(category**2) + 0.42262*category*device \
            + 0.19588*category*ad_type -8980.50169*(device**2) + 12626.37185*device*ad_type + 53.20869*(ad_type**2) \
            + 8.191084166028143e-09+ 1.75769292*error5

def to_user_profile(RT, SK, UA, ad_type, creative, country, exchange, error2):
    return -6.216e-07 + 9.83*RT -6.6*SK -3.23*UA -1.32*ad_type -0.0006*creative -0.86*country -6.2778e-06*exchange + 9.8*(RT**2) -3e-10*RT*SK -1e-10*RT*UA -9.23*RT*ad_type -0.0016*RT*creative + 11.55*RT*country -4.9848e-06*RT*exchange -6.6*(SK**2) + 3.3*SK*ad_type + 0.00059*SK*creative -3.7*SK*country + 1.4834e-06*SK*exchange -3.23*(UA**2) + 4.6*UA*ad_type + 0.0005*UA*creative -8.7*UA*country -2.7679e-06*UA*exchange -1.32*(ad_type**2) -3.35407e-05*ad_type*creative -5.7*ad_type*country + 5.4636e-06*ad_type*exchange -1e-10*(creative**2) + 7.60864e-05*creative*country + 1e-10*creative*exchange -0.86*(country**2) + 2.13998e-05*country*exchange -0.0*(exchange**2) + 2.020344363166225e-11 + 0.97971552*error2

def to_goal(exchange, country):
    return -7.27945e-05*exchange + 35.4976*country + 1e-10*(exchange**2) -9.24829e-05*exchange*country + 35.5*(country**2)

def to_probability(category, ad_type, user_profile, day_part, country, connection_type, exchange):
     return 3.6e-09 -3.4e-09*category -0.0017*ad_type -4.2267e-06*user_profile -8.96551e-05*day_part -0.00076*country -3.22984e-05*connection_type + 1e-10*exchange + 0.0*(category**2) + 2.6e-09*category*ad_type + 0.0*category*user_profile + 0.0*category*day_part + 1e-10*category*country + 1e-10*category*connection_type -0.0*category*exchange -0.00173*(ad_type**2) + 3.1211e-06*ad_type*user_profile + 4.27544e-05*ad_type*day_part + 0.00163*ad_type*country + 0.00035*ad_type*connection_type -7e-10*ad_type*exchange + 1.7e-09*(user_profile**2) + 2.27e-08*user_profile*day_part + 1.9398e-06*user_profile*country -2.991e-07*user_profile*connection_type -0.0*user_profile*exchange + 5.0122e-06*(day_part**2) + 3.3536e-06*day_part*country + 1.3187e-05*day_part*connection_type + 0.0*day_part*exchange -0.00076*(country**2) -7.76586e-05*country*connection_type -4e-10*country*exchange -9.68951e-05*(connection_type**2) -1e-10*connection_type*exchange + 0.0*(exchange**2)
    
def to_bid_price(goal, budget, ad_type, day_part, exchange, country, probability, app_views_count):
    return  -0.004 -0.037*goal -0.003*budget -9.882*ad_type -0.593*day_part -0.0*exchange -11.343*country -770.665*probability + 0.0*(goal**2) -0.0*goal*budget -0.007*goal*ad_type + 0.001*goal*day_part -0.0*goal*exchange + 0.051*goal*country + 2.318*goal*probability + 0.0*(budget**2) -0.001*budget*ad_type + 0.0*budget*day_part + 0.0*budget*exchange + 0.001*budget*country + 0.512*budget*probability -9.863*(ad_type**2) + 0.5*ad_type*day_part + 0.0*ad_type*exchange + 10.304*ad_type*country + 553.419*ad_type*probability -0.128*(day_part**2) + 0.0*day_part*exchange + 0.696*day_part*country -20.468*day_part*probability -0.0*(exchange**2) + 0.0*exchange*country + 0.001*exchange*probability -11.342*(country**2) + 826.853*country*probability + 491.348*(probability**2) -2.049786790812473e-11 + 0.99235287*app_views_count

def to_impressions(bid_price, day_part, user_profile, ad_type, country, creative, category, exchange):
     return(-5.5663 + 6.9396*bid_price -0.0954*day_part + 6.7361*user_profile + 1.2766*ad_type -0.0780*country + 0.0173*creative + 0.8825*category -0.9427*exchange)
    
def to_click(exchange, creative, ad_type, country, user_profile, probability, impressions, ):
    return (-1.9162808 + 0.4774666*exchange -1.6800447*creative -5.0000760*ad_type -0.3616471*country -4.5300518*user_profile + 11.2189913*probability + 4.6414798*impressions)

def to_installs(category, creative, ad_type, country, user_profile, probability, impressions, clicks, exchange):
    return (-6.3968 -2.0413*category -0.0523*impressions -3.6981*user_profile + 11.1342*probability + 14.8660*clicks -3.8073*creative -1.1441*country -0.7202*exchange)

def to_purchases(ad_type, installs, country, clicks, category, device, user_profile, creative,impressions, probability, exchange):
    return (-13.1807 -0.8743*ad_type + 2.9148*installs -0.2362*country + 0.0125*clicks -0.4308*category + 1.1486*device -0.0320*user_profile -0.2640*creative -0.0001*impressions + 0.0067*probability -0.2261*exchange)

def to_revenue(budget, purchases, bid_price, country, exchange, category):
    return 3.6e-09 -3.4e-09*budget -0.0017334935*purchases -4.2267e-06*bid_price -8.96551e-05*country -0.0007567844*exchange -3.22984e-05*category +  1e-10*(budget**2) + 0.0*budget*purchases + 2.6e-09*budget*bid_price + 0.0*budget*country + 0.0*budget*exchange + 1e-10*budget*category +1e-10*(purchases**2) -0.0*purchases*bid_price -0.0017*purchases*country + 3.1211e-06*purchases*exchange + 4.27544e-05*purchases*category + 0.0016*(bid_price**2) + 0.00035*bid_price*country -7e-10*bid_price*exchange + 1.7e-09*bid_price*category + 2.27e-08*(country**2) + 1.9398e-06*country*exchange -2.991e-07*country*category  -0.0*(exchange**2) + 5.0122e-06*exchange*category + 3.3536e-06*(category**2)

In [5]:
scm = FunctionalCausalModel({
    Functor(lambda: pyro.sample("country", dist.Categorical(probs=torch.tensor([0.5,0.5]))), 'country'),
    Functor(lambda: pyro.sample("category", dist.Categorical(probs=torch.tensor([0.5,0.5]))), 'category'),
    Functor(lambda: pyro.sample("RT", dist.Bernoulli(probs=torch.tensor(0.5))), 'RT'),
    Functor(lambda: pyro.sample("SK", dist.Bernoulli(probs=torch.tensor(0.5))), 'SK'),
    Functor(lambda: pyro.sample("UA", dist.Bernoulli(probs=torch.tensor(0.5))), 'UA'),
    Functor(lambda country: pyro.sample("connection_type", dist.Bernoulli(logits=(country_to_con_t_logits(country)))), 'connection_type'),
    Functor(lambda country: pyro.sample("day_part", dist.Bernoulli(logits=(country_to_day_p_logits(country)))), 'day_part'),
    Functor(lambda country, connection_type: pyro.sample("device", dist.Bernoulli(logits=(to_device(country, connection_type)))), 'device'),
    Functor(lambda device, day_part, country: pyro.sample("exchange", dist.Normal(to_exchange(device, day_part, country), std)), 'exchange'),
    Functor(lambda country, exchange: pyro.sample("budget", dist.Normal(to_budget(country, exchange), std)), 'budget'),
    Functor(lambda category, connection_type, exchange: pyro.sample("ad_type", dist.Bernoulli(logits=to_ad_type(category, connection_type, exchange))), 'ad_type'),
    Functor(lambda category, device, ad_type, error5: pyro.sample("creative", dist.Normal(to_creative(category, device, ad_type, error5), std)), 'creative'),
    Functor(lambda RT, SK, UA, ad_type, creative, country, exchange, error2: pyro.sample("user_profile", dist.Normal(to_user_profile(RT, SK, UA, ad_type, creative, country, exchange, error2), std)), 'user_profile'),
    Functor(lambda exchange, country: pyro.sample("goal", dist.Normal(to_goal(exchange, country), std)), 'goal'),
    Functor(lambda category, ad_type, user_profile, day_part, country, connection_type, exchange: pyro.sample("probability", dist.Normal(to_probability(category, ad_type, user_profile, day_part, country, connection_type, exchange), std)), 'probability'),
    Functor(lambda goal, budget, ad_type, day_part, exchange, country, probability, app_views_count: pyro.sample("bid_price", dist.Normal(to_bid_price(goal, budget, ad_type, day_part, exchange, country, probability, app_views_count), std)), 'bid_price'),
    Functor(lambda bid_price, day_part, user_profile, ad_type, country, creative, category, exchange: pyro.sample("impressions", dist.Bernoulli(logits=to_impressions(bid_price, day_part, user_profile, ad_type, country, creative, category, exchange))), 'impressions'),
    Functor(lambda exchange, creative, ad_type, country, user_profile, probability, impressions: pyro.sample("clicks", dist.Bernoulli(logits=to_click(exchange, creative, ad_type, country, user_profile, probability, impressions))), 'clicks'),
    Functor(lambda category, creative, ad_type, country, user_profile, probability, impressions, clicks, exchange: pyro.sample("installs", dist.Bernoulli(logits=to_installs(category, creative, ad_type, country, user_profile, probability, impressions, clicks, exchange))), 'installs'),
    Functor(lambda category, creative, ad_type, country, user_profile, probability, impressions, clicks, exchange: pyro.sample("purchases", dist.Bernoulli(logits=to_installs(category, creative, ad_type, country, user_profile, probability, impressions, clicks, exchange))), 'purchases'),
    Functor(lambda ad_type, installs, country, clicks, category, device, user_profile, creative,impressions, probability, exchange: pyro.sample("purchases", dist.Bernoulli(logits=to_purchases(ad_type, installs, country, clicks, category, device, user_profile, creative,impressions, probability, exchange))), 'purchases'),
    Functor(lambda budget, purchases, bid_price, country, exchange, category: pyro.sample("revenue", dist.Normal(to_revenue(budget, purchases, bid_price, country, exchange, category), std)), 'revenue'),

}
    , latents)

In [13]:
a = [scm.sample() for i in range(1_000)]

In [38]:
pd.DataFrame(a).goal.max()

tensor(74.3662)

In [39]:
from pomis.scm import Domain, RealDomain, BoolDomain

domain = [BoolDomain('ad_type'), RealDomain('budget', 10, 1100),
          RealDomain('creative', -17000, 100), RealDomain('goal', -1, 74)]

In [40]:
from experiments.causal_bo_experiment import CaBOExperiment, OptimizationObjective
non_int = (scm.endogenous - {d.name for d in domain})
smoke_test=True

In [41]:
non_int

{'RT',
 'SK',
 'UA',
 'bid_price',
 'category',
 'clicks',
 'connection_type',
 'country',
 'day_part',
 'device',
 'exchange',
 'impressions',
 'installs',
 'probability',
 'purchases',
 'revenue',
 'user_profile'}

In [50]:
scm.induced_graph().topological_order(include_uc=True)

NotImplementedError: 

In [43]:
cabo = CaBOExperiment(scm, non_int,
                         domain,  debug=True, target="revenue", n_iter=16 if smoke_test else n_iter,
                         experiment_name='test_ad', objetive=OptimizationObjective.maximize)

AssertionError: 